In [ ]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.5 MB/s eta 0:00:00


#Define your AWS access key and secret key

In [ ]:
import boto3
import os


#Creating bucket to store Input streaming data

In [ ]:
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID,
                         aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

# Define your AWS region
AWS_REGION = 'ap-south-1'  # Replace with your desired region

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID,aws_secret_access_key=AWS_SECRET_ACCESS_KEY,region_name=AWS_REGION)

# Create a bucket
bucket_name = 'raw-data-input1'
s3.create_bucket(Bucket=bucket_name,
                 CreateBucketConfiguration={
                     'LocationConstraint': AWS_REGION
                 })

{'ResponseMetadata': {'RequestId': 'J9FWF3J0J33MHPBF',
  'HostId': 'enjARXJnH5h4vIoNN/7O6LMx3ZuIZnk97TTTeScfI5PLvSa1sVBhmGz2nHk0j4kOFdgkSCBC8eqwtwo5WdIV+A==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'enjARXJnH5h4vIoNN/7O6LMx3ZuIZnk97TTTeScfI5PLvSa1sVBhmGz2nHk0j4kOFdgkSCBC8eqwtwo5WdIV+A==',
   'x-amz-request-id': 'J9FWF3J0J33MHPBF',
   'date': 'Sat, 17 Aug 2024 08:04:59 GMT',
   'location': 'http://raw-data-input1.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://raw-data-input1.s3.amazonaws.com/'}

#Creating Streaming Data


In [ ]:
!split -l 200 "/content/New 1000 Sales Records.txt" click_stream_

#Loading raw data to the S3 bucket

In [ ]:
import sys
import time
import glob
s3_resource = boto3.resource('s3',region_name='ap-south-1')
print("Uploading a file ....")
bucket_name= "raw-data-input1"
def func(a,b):
  local_file_name = a
  s3_file_name=b

  s3_resource.Bucket(bucket_name).upload_file(Filename=local_file_name, Key=s3_file_name)
  print("File upload completed!")
file_name=glob.glob('click_stream*')
file_path=[os.path.abspath(filename) for filename in file_name]

for i,j in zip(file_path,file_name):
  # time.sleep(5)
  func(i,j)

Uploading a file ....
File upload completed!
File upload completed!
File upload completed!
File upload completed!
File upload completed!
File upload completed!


#Creating S3 bucket for storing output

In [ ]:
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID,
                         aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

# Define your AWS region
AWS_REGION = 'ap-south-1'  # Replace with your desired region

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID,aws_secret_access_key=AWS_SECRET_ACCESS_KEY,region_name=AWS_REGION)

# Create a bucket
bucket_name = 'raw-data-output'
s3.create_bucket(Bucket=bucket_name,
                 CreateBucketConfiguration={
                     'LocationConstraint': AWS_REGION
                 })

{'ResponseMetadata': {'RequestId': '739F2BEGBXQANQB1',
  'HostId': 'pZkebxXgIK157HWenWuCbYD8R9VkAn1eG3ZwhxUh8OjeDHkJm5Lc7ROsAbc6MZCwgeiTkhKjUmZdjRGvYJKzKQ==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pZkebxXgIK157HWenWuCbYD8R9VkAn1eG3ZwhxUh8OjeDHkJm5Lc7ROsAbc6MZCwgeiTkhKjUmZdjRGvYJKzKQ==',
   'x-amz-request-id': '739F2BEGBXQANQB1',
   'date': 'Sat, 17 Aug 2024 08:45:55 GMT',
   'location': 'http://raw-data-output.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://raw-data-output.s3.amazonaws.com/'}

#AWS Lambda code to combining streaming data(AmazonS3FullAccess : add policy to lambda) + Add pandas layer

In [ ]:
import json
import boto3
import io
import pandas as pd

def lambda_handler(event, context):
    s3 = boto3.client("s3")
    response = s3.list_objects_v2(Bucket=bucket_name)

# Initialize an empty list to store file names
    list_input = []

# Check if the bucket has objects
    if 'Contents' in response:
      for item in response['Contents']:
        list_input.append(item['Key'])

    input_bucket = "raw-data-input"
    # list_input = ["aa", "ab","ac","ad","ae"]
    dataframes = []
    output_bucket = "raw-data-output"
    output_key = "combined_data.csv"

    for key in list_input:
        response = s3.get_object(Bucket=input_bucket, Key=key)
        csv_content = response['Body'].read().decode('utf-8')
        df = pd.read_csv(io.StringIO(csv_content),header=None)
        dataframes.append(df)

    if dataframes:
        combined_df = pd.concat(dataframes)

        #Data uploading part
        csv_buffer = io.StringIO()
        combined_df.to_csv(csv_buffer, index=False)
        csv_bytes = csv_buffer.getvalue().encode('utf-8')
        s3.put_object(Bucket=output_bucket, Key=output_key, Body=csv_bytes)

#Redshift (Load s3 bucket into redshift)

*   Load s3 bucket into table trial and make it public
*   GRANT SELECT ON TABLE "dev"."public"."trial" TO PUBLIC;



In [ ]:
pip install boto3 psycopg2

In [ ]:
import boto3
import psycopg2
import json
def func_query(sql_query):
# Initialize a session using Amazon Redshift Serverless
  session = boto3.Session(
      aws_access_key_id=AWS_ACCESS_KEY_ID,
      aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
      region_name='ap-south-1'
  )

# Get the Redshift Data API client
  client = session.client('redshift-data')

# Set the database name, workgroup name, and other required variables
  database = 'dev'
  workgroup_name = 'default-workgroup'  # Use workgroup name for Redshift Serverless
  db_user = 'admin'
  password = 'avi.bansal3A'
# sql_query = 'SELECT * FROM "dev"."public"."project_table" limit 10;'


# Execute the SQL query using WorkgroupName for Redshift Serverless
  response = client.execute_statement(
      Database=database,
      WorkgroupName=workgroup_name,  # Use WorkgroupName instead of ClusterIdentifier
    # DbUser=db_user,
      Sql=sql_query
  )

# Get the query ID from the response
  query_id = response['Id']

  while True:
      status = client.describe_statement(Id=query_id)
      if status['Status'] in ['STARTED', 'SUBMITTED']:
          continue
      elif status['Status'] in ['FINISHED', 'FAILED']:
          break

# Fetch the results
  if status['Status'] == 'FINISHED':
      result = client.get_statement_result(Id=query_id)
      records = result['Records']
      for record in records:
          print(record)
  else:
      print("Query failed:", status['Status'])
      print(status)

#Selecting First 10 rows

In [ ]:
func_query("SELECT * FROM public.project_table limit 10;")

[{'stringValue': 'Middle East and North Africa'}, {'stringValue': 'Libya'}, {'longValue': 8446}]
[{'stringValue': 'North America'}, {'stringValue': 'Canada'}, {'longValue': 3018}]
[{'stringValue': 'Middle East and North Africa'}, {'stringValue': 'Libya'}, {'longValue': 1517}]
[{'stringValue': 'Asia'}, {'stringValue': 'Japan'}, {'longValue': 3322}]
[{'stringValue': 'Sub-Saharan Africa'}, {'stringValue': 'Chad'}, {'longValue': 9845}]
[{'stringValue': 'Europe'}, {'stringValue': 'Armenia'}, {'longValue': 9528}]
[{'stringValue': 'Sub-Saharan Africa'}, {'stringValue': 'Eritrea'}, {'longValue': 2844}]
[{'stringValue': 'Europe'}, {'stringValue': 'Montenegro'}, {'longValue': 7299}]
[{'stringValue': 'Central America and the Caribbean'}, {'stringValue': 'Jamaica'}, {'longValue': 2428}]
[{'stringValue': 'Australia and Oceania'}, {'stringValue': 'Fiji'}, {'longValue': 4800}]


#Selecting Regions

In [ ]:
func_query("SELECT region FROM public.project_table limit 10;")

[{'stringValue': 'Middle East and North Africa'}]
[{'stringValue': 'North America'}]
[{'stringValue': 'Middle East and North Africa'}]
[{'stringValue': 'Asia'}]
[{'stringValue': 'Sub-Saharan Africa'}]
[{'stringValue': 'Europe'}]
[{'stringValue': 'Sub-Saharan Africa'}]
[{'stringValue': 'Europe'}]
[{'stringValue': 'Central America and the Caribbean'}]
[{'stringValue': 'Australia and Oceania'}]


#Top 5 country by sales

In [ ]:
func_query("SELECT sales,country FROM public.project_table order by sales desc limit  5;")

[{'longValue': 9998}, {'stringValue': 'Guinea-Bissau'}]
[{'longValue': 9980}, {'stringValue': 'New Zealand'}]
[{'longValue': 9958}, {'stringValue': 'Luxembourg'}]
[{'longValue': 9951}, {'stringValue': 'Serbia'}]
[{'longValue': 9950}, {'stringValue': 'Andorra'}]


#Sales in Every Region

In [ ]:
func_query("SELECT region,sum(sales) FROM public.project_table group by region;")

[{'stringValue': 'Middle East and North Africa'}, {'longValue': 682363}]
[{'stringValue': 'North America'}, {'longValue': 100739}]
[{'stringValue': 'Asia'}, {'longValue': 677524}]
[{'stringValue': 'Sub-Saharan Africa'}, {'longValue': 1386894}]
[{'stringValue': 'Europe'}, {'longValue': 1285808}]
[{'stringValue': 'Central America and the Caribbean'}, {'longValue': 503362}]
[{'stringValue': 'Australia and Oceania'}, {'longValue': 417298}]


#Country Count in every region

In [ ]:
func_query("SELECT region,count(distinct country) FROM public.project_table group by region;")

[{'stringValue': 'Middle East and North Africa'}, {'longValue': 23}]
[{'stringValue': 'North America'}, {'longValue': 4}]
[{'stringValue': 'Asia'}, {'longValue': 27}]
[{'stringValue': 'Sub-Saharan Africa'}, {'longValue': 48}]
[{'stringValue': 'Europe'}, {'longValue': 48}]
[{'stringValue': 'Central America and the Caribbean'}, {'longValue': 20}]
[{'stringValue': 'Australia and Oceania'}, {'longValue': 15}]


#Country Sales > average sales

In [ ]:
func_query("SELECT distinct (Country) FROM public.project_table where sales >= (select avg(sales) FROM public.project_table) limit 10;")

[{'stringValue': 'Libya'}]
[{'stringValue': 'Chad'}]
[{'stringValue': 'Armenia'}]
[{'stringValue': 'Montenegro'}]
[{'stringValue': 'Maldives'}]
[{'stringValue': 'Senegal'}]
[{'stringValue': 'Bulgaria'}]
[{'stringValue': 'Algeria'}]
[{'stringValue': 'Grenada'}]
[{'stringValue': 'Mauritius '}]
